# Respostas dos exercícios da aula 05

In [14]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re

from IPython.display import display

## Carregando dados salvos

In [6]:
dadosCelulares = pd.read_csv('../2017.1/data/scraping/dadosConsolidadosIPhone7.csv', header=0)
dadosCelulares.head()

,modelo,capacidade,vendedor,preco,link
0,iPhone 6S,128,Mercado Livre,3399.00,http://www.buscape.com.br/redirect_prod?pagina...
1,iPhone 6S,128,Fnac.com,3059.10,http://www.buscape.com.br/redirect_prod?pagina...
2,iPhone 6S,128,Saraiva.com.br,3229.05,http://www.buscape.com.br/redirect_prod?pagina...
3,iPhone 6S,128,Carrefour,3999.00,http://www.buscape.com.br/redirect_prod?pagina...
4,iPhone 6S,128,Casas Bahia,4599.00,http://www.buscape.com.br/redirect_prod?pagina...


## Respostas dos exercícios

2. Escreva trechos de código para responder às seguintes perguntas:
  1. Qual o preço médio de cada modelo de iPhone (independente da capacidade ou cor)?
  2. Qual é a loja mais barata em média?
  3. Qual modelo possui a maior diferença de preço?
  4. Qual a variação de preço por modelo e capacidade?
1. Modifique a parte de coleta dos dados da página inicial da busca para também armazenar o número de avaliações feitas ao produto e a nota média exibida (número de estrelas)  

#### Exercício 1

In [7]:
# 1a
dcGroupedByModelo = dadosCelulares.groupby('modelo')
np.round(dcGroupedByModelo.preco.mean(),2)

modelo
iPhone 5S        1649.49
iPhone 6         2833.39
iPhone 6S        3132.35
iPhone 7         3594.29
iPhone 7 Plus    4129.39
Name: preco, dtype: float64

In [8]:
# 1b
dcGroupedByLoja = dadosCelulares.groupby('vendedor')
np.argmin(dcGroupedByLoja.preco.mean())

'DouGShoP'

In [9]:
# 1c
np.argmax(dcGroupedByModelo.preco.max() - dcGroupedByModelo.preco.min())

'iPhone 6S'

In [16]:
dcGroupedByModelCap = dadosCelulares.groupby(['modelo','capacidade'])
dcGroupedByModelCap.preco.agg(lambda x: np.round(((max(x) - min(x))/min(x))*100,2))

modelo         capacidade
iPhone 5S      16             44.72
               32              0.00
iPhone 6       16            122.18
               64             35.07
               128             0.00
iPhone 6S      16             49.54
               32             11.11
               64             78.67
               128            53.76
iPhone 7       32             26.21
               128            32.39
               256            40.06
iPhone 7 Plus  32             17.15
               128            16.89
               256            16.67
Name: preco, dtype: float64

#### Exercício 2

In [21]:
url = "../2017.1/data/scraping/buscape/7"
parsed_page = BeautifulSoup(open(url,encoding="latin1"),"html.parser")

precoToFloat = lambda x: float(re.sub(',','.',re.sub('[a-zA-Z\$\s\.]+','',x)))
listCelulares = []
for a in parsed_page.find_all('li',itemtype=re.compile('Product')):
    # Parte identica aa aula
    modelo, capacidade = re.match('(iPhone \dS?(?:\sPlus)?)\s*(\d+)GB',\
                                  re.sub('Smartphone Apple ','',\
                                         a.find('div',class_='bui-product__name').text)).groups()
    menorPreco = precoToFloat(a.find(itemprop='lowPrice').text)
    maiorPreco = precoToFloat(a.find(itemprop='highPrice').text)
    disponibilidade = int(a.find('span',attrs={'name': 'offerCount'}).text)
    
    # Modificacoes do exercicio
    
    num_avaliacoes = int(re.sub('\.','',a.find('span', itemprop='ratingCount').text))    
    avaliacao = 1.0 * len(a.find_all('span',class_='starfull')) + \
                        0.5 * len(a.find_all('span',class_='starhalf')) 
    
    listCelulares.append(dict(zip(['modelo','capacidade','menorPreco','maiorPreco','disponibilidade',
                                   'num_avaliacoes', 'avaliacao'],
                                  [modelo,int(capacidade),menorPreco,maiorPreco,disponibilidade, 
                                   num_avaliacoes,avaliacao])))

dataCelulares = pd.DataFrame(listCelulares,
                             columns=['modelo','capacidade','menorPreco','maiorPreco','disponibilidade',
                                     'num_avaliacoes', 'avaliacao'])
print(dataCelulares.describe())
dataCelulares

       capacidade   menorPreco   maiorPreco  disponibilidade  num_avaliacoes  \
count   15.000000    15.000000    15.000000        15.000000       15.000000   
mean    88.533333  2990.282000  3934.018667         9.866667      133.200000   
std     81.104753   779.716991   920.101374         4.596065      262.436986   
min     16.000000  1381.250000  2149.000000         1.000000        4.000000   
25%     32.000000  2385.360000  3323.085000         7.000000       10.500000   
50%     64.000000  2991.120000  3999.000000        12.000000       26.000000   
75%    128.000000  3499.000000  4599.000000        13.000000       97.500000   
max    256.000000  4199.000000  5305.000000        15.000000     1006.000000   

       avaliacao  
count  15.000000  
mean    4.733333  
std     0.319970  
min     4.000000  
25%     4.500000  
50%     5.000000  
75%     5.000000  
max     5.000000  


,modelo,capacidade,menorPreco,maiorPreco,disponibilidade,num_avaliacoes,avaliacao
0,iPhone 5S,16,1381.25,2559.99,15,1006,4.5
1,iPhone 7,128,3399.00,3959.12,14,25,4.5
2,iPhone 7,32,2931.65,3607.12,14,12,5.0
3,iPhone 7 Plus,128,3849.00,4499.00,13,15,4.0
4,iPhone 7 Plus,256,4199.00,4899.00,13,9,5.0
5,iPhone 6S,128,2991.12,4599.00,12,26,4.5
6,iPhone 7 Plus,32,3499.00,4099.00,12,9,4.5
7,iPhone 7,256,3497.75,4899.00,12,7,5.0
8,iPhone 6,16,2249.10,4599.00,11,396,5.0
9,iPhone 6S,16,2464.41,3999.00,11,64,5.0
